In [1]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import shared_utils

import data_prep
import _utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You mi

In [2]:
import altair as alt
import altair_saver
from shared_utils import geography_utils
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [3]:
pd.set_option("display.max_columns", 100)

### Read in BC_GTFS_NTD_2

In [4]:
df= pd.read_parquet('BC_GTFS_NTD_2.parquet')

In [5]:
df.sample(5)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size,adjusted_allocationamount,adjusted_encumbered_amount,adjusted_expendedamount,adjusted_activebalance,adjusted_closedoutbalance,_16plus,name,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,year
65,2016,Section 5311,CA-2017-025 | 0017000158,2016,San Mateo County Transit District,BCG0000107,Operating Assistance,300902,64BO17-00427,148815.0,148815.0,148815.0,0.0,0,Open,90009,Full Reporter,305.0,171.0,0.0,102.0,0.0,0.0,0.0,0.0,578.0,7.201646,168268.7586,NaN,11.0,401.0,0.0,82.0,0.0,84.0,22.0,802.0,0.0,84.0,908.0,290.0,0,Ok,Large,168013.327535,168013.327535,168013.327535,0.000000,0.0,102.0,San Mateo County Transit District,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static Incomplete,RT Incomplete,2016-01-01
640,2018,Section 5311,CA-2018-114 | 0019000012,2018,Tuolumne County Transit,BCG0001132,Operating Assistance Sliding Scale,300902,64BO19-00879,305527.0,305527.0,305527.0,0.0,0,Open,9R02-91057,Rural Reporter,27.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,30.0,6.681548,0.0000,NaN,0.0,8.0,0.0,1.0,0.0,21.0,0.0,16.0,0.0,21.0,37.0,482.0,0,Needed,Medium,329694.918550,329694.918550,329694.918550,0.000000,0.0,1.0,None,None,None,None,None,None,2018-01-01
721,2020,Section 5311,CA-2021-030 | 0020000131,2020,County of Shasta Department of Public Works,BCG0002395,Operating Assistance Sliding Scale,300902,64BO20-01419,160881.0,360153.0,360153.0,-199272.0,0,Open,9R02-99438,Asset Subrecipient,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.000000,0.0000,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0,Ok,Small,168439.126066,377072.846204,377072.846204,-208633.720137,0.0,0.0,None,None,None,None,None,None,2020-01-01
394,2020,Section 5311,CA-2021-030 | 0020000131,2020,Palo Verde Valley Transit Agency,BCG0002754,Purchase Replacement < 30 Ft Bus,111204,64BC20-01385,101073.0,101073.0,0.0,101073.0,0,Open,9R02-99454,Rural Reporter,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,4.090909,0.0000,NaN,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,11.0,11.0,238.0,0,Ok,Medium,105821.369763,105821.369763,0.000000,105821.369763,0.0,0.0,Palo Verde Valley Transit Agency,08 - San Bernardino,Southern California Association of Governments,Southern California Association of Governments,Static OK,RT Incomplete,2020-01-01
374,2020,Section 5311,CA-2021-030 | 0020000131,2020,Eastern Sierra Transit Authority,BCG0002408,Operating Assistance Sliding Scale - Inyo,300902,64BO20-01422,112274.0,112274.0,112274.0,0.0,0,Open,9R02-91062,Rural Reporter,73.0,16.0,8.0,0.0,0.0,0.0,0.0,0.0,97.0,7.305816,0.0000,NaN,0.0,41.0,0.0,10.0,0.0,46.0,0.0,82.0,0.0,46.0,128.0,99.0,0,Ok,Large,117548.588335,117548.588335,117548.588335,0.000000,0.0,0.0,Eastern Sierra Transit Authority,09 - Bishop,None,Mono County Local Transportation Commission,Static OK,RT OK,2020-01-01


In [9]:
df>>count(_.caltrans_district)

,caltrans_district,n
0,01 - Eureka,75
1,02 - Redding,29
2,03 - Marysville,71
3,04 - Oakland,49
4,05 - San Luis Obispo,56
5,06 - Fresno,107
6,07 - Los Angeles,10
7,08 - San Bernardino,78
8,09 - Bishop,36
9,10 - Stockton,31


### Add Airtable 

In [17]:
# GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/5311 /"
# File_Organization =  "organizations-AllOrganizations_1.csv"
# organizations =  pd.read_csv(f'{GCS_FILE_PATH}{File_Organization}')
# airtable = to_snakecase(organizations)

# airtable = airtable>>select(_.name,
#                       # _.ntp_id,
#                        # _.itp_id,
#                       _.caltrans_district,
#                       _.mpo_rtpa,
#                       _.planning_authority,
#                       _.gtfs_static_status,
#                       _.gtfs_realtime_status
#                      )    

In [18]:
#df = pd.merge(df, airtable, how='left', left_on='organization_name', right_on='name')

In [19]:
#df = df.drop('name', 1)

In [13]:
#df.to_parquet("BC_GTFS_NTD_2.parquet")

### Read in 5311 Agg 

In [6]:
df1 = pd.read_parquet('5311_agg.parquet')

In [11]:
df1.head()

,organization_name,reporter_type,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status
0,Madera County,Rural Reporter,Medium,9R02-91005,188.0,150308.833333,1.567981e+05,110124.740556,40184.092778,0.0,162562.362848,120648.235973,1.726299e+05,41914.126876,0.0,16.0,5.464286,0.0000,0.0,0.0,0.0,0.0,16.0,0.0,0.0,16.0,0.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,06 - Fresno,None,None,Static OK,RT OK
1,Fresno County Rural Transit Agency,Rural Reporter,Large,9R02-91007,117.0,877618.700000,1.401935e+06,631136.300000,254384.200000,0.0,932754.892597,687760.790181,1.528561e+06,253915.297938,0.0,178.0,5.488889,0.0000,6.0,21.0,0.0,0.0,148.0,12.0,42.0,148.0,0.0,202.0,117.0,58.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,06 - Fresno,Fresno Council of Governments,Fresno Council of Governments,Static Incomplete,RT Incomplete
2,Yuba-Sutter Transit Authority,Full Reporter,Medium,90061,376.0,89890.214286,1.358760e+05,52671.892857,37218.321429,0.0,96563.322274,56918.842033,1.495202e+05,39644.480242,0.0,52.0,4.035256,134500.6572,0.0,35.0,0.0,0.0,16.0,0.0,70.0,16.0,0.0,86.0,48.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,03 - Marysville,Sacramento Area Council of Governments,Sacramento Area Council of Governments,Static OK,RT Incomplete
3,City of Arcata,Rural Reporter,Medium,9R02-91018,18.0,82111.428571,1.435400e+05,82111.428571,0.000000,0.0,89821.087756,89821.087756,1.586698e+05,0.000000,0.0,10.0,8.500000,0.0000,0.0,5.0,0.0,0.0,2.0,0.0,10.0,2.0,0.0,12.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,01 - Eureka,None,Humboldt County Association of Governments,Static OK,RT OK
4,City of Ojai,Rural Reporter,Small,9R02-91058,231.0,270678.200000,2.263345e+05,216975.417000,53702.783000,0.0,290756.167674,236530.491469,2.464124e+05,54225.676205,0.0,7.0,7.500000,0.0000,0.0,6.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,12.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,07 - Los Angeles,Southern California Association of Governments,Southern California Association of Governments,Static OK,RT OK


## District 3

In [10]:
df_all = df>>filter(_.caltrans_district=='11 - San Diego')
df_all.head()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size,adjusted_allocationamount,adjusted_encumbered_amount,adjusted_expendedamount,adjusted_activebalance,adjusted_closedoutbalance,_16plus,name,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,year
15,2014,Section 5311,CA-18-X069 | 0015000075,2017,San Diego Metropolitan Transit System,BCG0001128,Operating Assistance Sliding Scale,300902,64BO17-00579,131388.0,275000.0,131388.0,0.0,0,Open,90026,Full Reporter,757.0,156.0,40.0,3.0,53.0,1.0,1.0,2.0,1013.0,19.713625,378469.187667,60.0,0.0,598.0,0.0,27.0,164.0,224.0,0.0,1196.0,328.0,224.0,1748.0,278.0,0,Ok,Large,145244.215989,304001.578507,145244.215989,0.0,0.0,58.0,San Diego Metropolitan Transit System,11 - San Diego,San Diego Association of Governments,San Diego Association of Governments,Static OK,RT Incomplete,2017-01-01
47,2016,Section 5311,CA-2017-025 | 0017000158,2016,Imperial County Transportation Commission,BCG0000183,Operating Assistance,300902,64BO17-00402,64841.0,64841.0,64841.0,0.0,0,Open,90226,Full Reporter,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,4.227564,142255.118600,NaN,0.0,16.0,0.0,3.0,0.0,27.0,0.0,32.0,0.0,27.0,59.0,138.0,0,Needed,Medium,73206.008606,73206.008606,73206.008606,0.0,0.0,0.0,Imperial County Transportation Commission,11 - San Diego,Southern California Association of Governments,Southern California Association of Governments,Static Incomplete,RT Incomplete,2016-01-01
48,2016,Section 5311,CA-2017-025 | 0017000158,2016,Imperial County Transportation Commission,BCG0000185,Operating Assistance,300902,64BO17-00402,32293.0,32293.0,32293.0,0.0,0,Open,90226,Full Reporter,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,4.227564,142255.118600,NaN,0.0,16.0,0.0,3.0,0.0,27.0,0.0,32.0,0.0,27.0,59.0,138.0,0,Needed,Medium,36459.055781,36459.055781,36459.055781,0.0,0.0,0.0,Imperial County Transportation Commission,11 - San Diego,Southern California Association of Governments,Southern California Association of Governments,Static Incomplete,RT Incomplete,2016-01-01
49,2016,Section 5311,CA-2017-025 | 0017000158,2016,Imperial County Transportation Commission,BCG0000186,Operating Assistance,300902,64BO17-00402,164302.0,164302.0,164302.0,0.0,0,Open,90226,Full Reporter,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,4.227564,142255.118600,NaN,0.0,16.0,0.0,3.0,0.0,27.0,0.0,32.0,0.0,27.0,59.0,138.0,0,Needed,Medium,185498.274641,185498.274641,185498.274641,0.0,0.0,0.0,Imperial County Transportation Commission,11 - San Diego,Southern California Association of Governments,Southern California Association of Governments,Static Incomplete,RT Incomplete,2016-01-01
59,2016,Section 5311,CA-2017-025 | 0017000158,2016,North County Transit District,BCG0000154,Operating Assistance,300902,64BO17-00417,411177.0,411177.0,411177.0,0.0,0,Open,90030,Full Reporter,168.0,22.0,90.0,73.0,6.0,21.0,0.0,0.0,380.0,14.194925,525021.420533,100.0,0.0,152.0,0.0,123.0,44.0,61.0,0.0,304.0,88.0,61.0,453.0,226.0,0,Ok,Large,464222.127985,464222.127985,464222.127985,0.0,0.0,100.0,North County Transit District,11 - San Diego,San Diego Association of Governments,San Diego Association of Governments,Static OK,RT OK,2016-01-01


In [15]:
len(df_all)

25

In [23]:
df2 = df1>>filter(_.caltrans_district=='11 - San Diego')>>arrange(-_.adjusted_allocationamount)
df2

,organization_name,reporter_type,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status
37,North County Transit District,Full Reporter,Large,90030,226.0,440263.500000,359623.833333,359623.833333,80639.666667,0.0,470048.610094,389408.874751,389408.874751,80639.735343,0.0,380.0,14.194925,525021.420533,0.0,152.0,0.0,44.0,61.0,0.0,304.0,61.0,88.0,453.0,168.0,22.0,90.0,73.0,6.0,21.0,0.0,100.0,0.0,11 - San Diego,San Diego Association of Governments,San Diego Association of Governments,Static OK,RT OK
5,San Diego Metropolitan Transit System,Full Reporter,Large,90026,278.0,240955.727273,265955.727273,162398.154545,78557.572727,0.0,259434.104329,176254.422231,287070.611466,83179.682099,0.0,1013.0,19.713625,378469.187667,0.0,598.0,0.0,164.0,224.0,0.0,1196.0,224.0,328.0,1748.0,757.0,156.0,40.0,3.0,53.0,1.0,1.0,58.0,2.0,11 - San Diego,San Diego Association of Governments,San Diego Association of Governments,Static OK,RT Incomplete
27,Imperial County Transportation Commission,Full Reporter,Medium,90226,138.0,229595.250000,177860.875000,153779.750000,75815.500000,0.0,244525.805681,167332.204679,192791.386621,77193.601001,0.0,46.0,4.227564,142255.118600,0.0,16.0,0.0,0.0,27.0,0.0,32.0,27.0,0.0,59.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 - San Diego,Southern California Association of Governments,Southern California Association of Governments,Static Incomplete,RT Incomplete


In [13]:
df2.describe()

,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.0,3.000000,3.000000,3.000000,3.000000,3.0,3.000000,3.000000,3.000000,3.0,3.000000,3.0,3.000000,3.000000,3.0,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,214.000000,303604.825758,267813.478535,225267.245960,78337.579798,0.0,324669.506701,244331.833887,289756.957613,80337.672814,0.0,479.666667,12.712038,348581.908933,0.0,255.333333,0.0,69.333333,104.000000,0.0,510.666667,104.000000,138.666667,753.333333,323.666667,59.333333,43.333333,25.333333,19.666667,7.333333,0.333333,52.666667,0.666667
std,70.767224,118486.117521,90895.718740,116435.985212,2419.595767,0.0,126122.468734,125719.578153,98336.267477,3004.450541,0.0,491.143903,7.848805,193125.470994,0.0,304.449229,0.0,84.884235,105.304321,0.0,608.898459,105.304321,169.768470,883.646045,380.203016,84.435380,45.092498,41.307788,29.022979,11.846237,0.577350,50.212880,1.154701
min,138.000000,229595.250000,177860.875000,153779.750000,75815.500000,0.0,244525.805681,167332.204679,192791.386621,77193.601001,0.0,46.000000,4.227564,142255.118600,0.0,16.000000,0.0,0.000000,27.000000,0.0,32.000000,27.000000,0.000000,59.000000,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,182.000000,235275.488636,221908.301136,158088.952273,77186.536364,0.0,251979.955005,171793.313455,239930.999044,78916.668172,0.0,213.000000,9.211244,260362.153133,0.0,84.000000,0.0,22.000000,44.000000,0.0,168.000000,44.000000,44.000000,256.000000,107.000000,11.000000,20.000000,1.500000,3.000000,0.500000,0.000000,29.000000,0.000000
50%,226.000000,240955.727273,265955.727273,162398.154545,78557.572727,0.0,259434.104329,176254.422231,287070.611466,80639.735343,0.0,380.000000,14.194925,378469.187667,0.0,152.000000,0.0,44.000000,61.000000,0.0,304.000000,61.000000,88.000000,453.000000,168.000000,22.000000,40.000000,3.000000,6.000000,1.000000,0.000000,58.000000,0.000000
75%,252.000000,340609.613636,312789.780303,261010.993939,79598.619697,0.0,364741.357212,282831.648491,338239.743109,81909.708721,0.0,696.500000,16.954275,451745.304100,0.0,375.000000,0.0,104.000000,142.500000,0.0,750.000000,142.500000,208.000000,1100.500000,462.500000,89.000000,65.000000,38.000000,29.500000,11.000000,0.500000,79.000000,1.000000
max,278.000000,440263.500000,359623.833333,359623.833333,80639.666667,0.0,470048.610094,389408.874751,389408.874751,83179.682099,0.0,1013.000000,19.713625,525021.420533,0.0,598.000000,0.0,164.000000,224.000000,0.0,1196.000000,224.000000,328.000000,1748.000000,757.000000,156.000000,90.000000,73.000000,53.000000,21.000000,1.000000,100.000000,2.000000


In [16]:
df_all.describe()

,grant_fiscal_year,project_year,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,adjusted_allocationamount,adjusted_encumbered_amount,adjusted_expendedamount,adjusted_activebalance,adjusted_closedoutbalance,_16plus
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.0,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,17.000000,25.0,25.000000,25.0,25.000000,25.000000,25.000000,25.0,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.0,25.000000
mean,2018.040000,2018.160000,285154.240000,260245.720000,206974.428000,78179.812000,0.0,388.120000,73.920000,39.200000,18.840000,24.760000,5.480000,0.440000,0.880000,551.640000,13.433598,338053.221453,74.117647,0.0,304.720000,0.0,42.360000,82.720000,121.840000,0.0,609.440000,165.440000,121.840000,896.720000,220.720000,305210.930145,281462.442704,224556.381219,80654.548926,0.0,49.520000
std,1.813836,1.624808,124451.069647,132179.227441,152530.310902,143684.834244,0.0,336.890353,74.720546,34.024501,31.090835,25.648717,8.912164,0.506623,1.013246,436.058089,6.816990,149221.001514,19.703687,0.0,270.259061,0.0,47.436695,75.389831,93.311611,0.0,540.518122,150.779663,93.311611,784.404685,61.576186,129694.450714,142178.359882,164838.117161,146020.857512,0.0,38.567171
min,2014.000000,2016.000000,32293.000000,0.000000,0.000000,0.000000,0.0,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,46.000000,4.227564,142255.118600,60.000000,0.0,16.000000,0.0,3.000000,0.000000,27.000000,0.0,32.000000,0.000000,27.000000,59.000000,138.000000,36459.055781,0.000000,0.000000,0.000000,0.0,0.000000
25%,2017.000000,2017.000000,203500.000000,203500.000000,64841.000000,0.000000,0.0,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,46.000000,4.227564,142255.118600,60.000000,0.0,16.000000,0.0,3.000000,0.000000,27.000000,0.0,32.000000,0.000000,27.000000,59.000000,138.000000,219597.338124,219597.338124,73206.008606,0.000000,0.0,0.000000
50%,2018.000000,2018.000000,280178.000000,275000.000000,203500.000000,0.000000,0.0,168.000000,22.000000,40.000000,3.000000,6.000000,1.000000,0.000000,0.000000,380.000000,14.194925,378469.187667,60.000000,0.0,152.000000,0.0,27.000000,44.000000,61.000000,0.0,304.000000,88.000000,61.000000,453.000000,226.000000,304001.578507,304001.578507,219597.338124,0.000000,0.0,58.000000
75%,2019.000000,2019.000000,411177.000000,323827.000000,300000.000000,102801.300000,0.0,757.000000,156.000000,40.000000,3.000000,53.000000,1.000000,1.000000,2.000000,1013.000000,19.713625,378469.187667,100.000000,0.0,598.000000,0.0,27.000000,164.000000,224.000000,0.0,1196.000000,328.000000,224.000000,1748.000000,278.000000,413875.352473,339040.265019,317968.519166,113642.754446,0.0,58.000000
max,2021.000000,2021.000000,483838.000000,465994.000000,465994.000000,483838.000000,0.0,757.000000,156.000000,90.000000,73.000000,53.000000,21.000000,1.000000,2.000000,1013.000000,19.713625,525021.420533,100.000000,0.0,598.000000,0.0,123.000000,164.000000,224.000000,0.0,1196.000000,328.000000,224.000000,1748.000000,278.000000,487886.214730,487886.214730,487886.214730,483838.412057,0.0,100.000000


## Run Charting Functions 

In [14]:
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    LABEL_DICT = { "prepared_y": "Year",
              "dist": "District",
              "nunique":"Number of Unique",
              "project_no": "Project Number"}
    
    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        #word = word.replace('n_', 'Number of ').title()
        word = word.replace('unique_', "Number of Unique ").title()
        word = word.replace('_', ' ').title()
    
    return word


def basic_bar_chart(df, x_col, y_col, color_col, chart_title=''):
    
    if chart_title == "":
        chart_title = (f"{labeling(x_col)} by {labeling(y_col)}")
    
    chart = (alt.Chart(df)
             .mark_bar()
             .encode(
                 x=alt.X(x_col, title=labeling(x_col), sort=('-y')),
                 y=alt.Y(y_col, title=labeling(y_col)),
                 #column = "payment:N",
                 color = alt.Color(color_col,
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                                      legend=alt.Legend(title=(labeling(color_col)))
                                  ))
             .properties( 
                          title=chart_title)
    )

    chart=styleguide.preset_chart_config(chart)
    
    return chart

## Charts

In [17]:
basic_bar_chart(df2, 'organization_name', 'total_vehicles', 'organization_name', chart_title='Fleet Size by Agency')

alt.Chart(...)

In [18]:
basic_bar_chart(df2, 'organization_name', 'adjusted_allocationamount', 'organization_name', chart_title='Aveage Allocation Amount Size by Agency')

alt.Chart(...)

In [19]:
basic_bar_chart(df2, 'organization_name',
                'average_age_of_fleet__in_years_',
                'organization_name',
                chart_title='Average Fleet Age by Agency')

alt.Chart(...)

In [20]:
chart = (alt.Chart(df_all)
             .mark_bar(size=30)
             .encode(
                 x=alt.X('project_year:O', title=labeling('project_year')),
                 y=alt.Y('adjusted_allocationamount:Q', title=('Allocation Amount ($2021)')),
                 color = alt.Color('organization_name:N',
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                                      legend=alt.Legend(title=(labeling('organization_name')))
                                  )
             )
             .properties( 
                          title=('Allocation Amount by Year'))
    )

chart=styleguide.preset_chart_config(chart)
    
display(chart)

alt.Chart(...)

In [21]:
df_count = df_all>>group_by(_.project_year)>>count(_.organization_name)
df_count

,project_year,organization_name,n
0,2016,Imperial County Transportation Commission,3
1,2016,North County Transit District,1
2,2016,San Diego Metropolitan Transit System,1
3,2017,Imperial County Transportation Commission,1
4,2017,North County Transit District,1
5,2017,San Diego Metropolitan Transit System,3
6,2018,Imperial County Transportation Commission,1
7,2018,North County Transit District,1
8,2018,San Diego Metropolitan Transit System,2
9,2019,Imperial County Transportation Commission,1


In [22]:
chart = (alt.Chart(df_count)
             .mark_bar(size=30)
             .encode(
                 x=alt.X('project_year:O', title=labeling('project_year')),
                 y=alt.Y('n:Q', title=('Number of Projects')),
                 color = alt.Color('organization_name:N',
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                                      legend=alt.Legend(title=(labeling('organization_name')))
                                  )
    #             , column='organization_name:N'
             )
             .properties( 
                          title=('Number of Projects by Year'))
    )

chart=styleguide.preset_chart_config(chart)
    
display(chart)

alt.Chart(...)